#TF-IDF

---

##Importing necessary Libraries

In [ ]:
# requirements
!pip install num2words

In [4]:
import re
import nltk
import num2words
import pandas as pd;
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Tokenizer

In [5]:
porter_stemmer = nltk.stem.porter.PorterStemmer()


def tokenize_Words(sentences):
    punctuations=['+',',','?','!','/','@','#','$']
    tokens=[]
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)	
        words = map(lambda x:x.lower(),words)
        filtered_words = [word for word in words if word not in nltk.corpus.stopwords.words('english')+punctuations]
        stemmed_word=[porter_stemmer.stem(word) for word in filtered_words]
        add_numbers=[num2words.num2words(int(words)) if words.isnumeric() else words for words in stemmed_word]
        tokens.append(add_numbers)
    return tokens



## Import Dataset
> Source: [Cricket Commentary - Kaggle](https://www.kaggle.com/shineucc/cricket-commentary)

In [7]:
Dataframe = pd.read_csv('dataset.csv')

Dataframe.head()

,Unnamed: 0,description,tags
0,0,wankhedes notorious reputation for being a cha...,sports mumbai rahane ball bat chasing over thi...
1,1,pandya to rahane no run picks a short of lengt...,sports length delivery delivery leg short wris...
2,2,harbhajan to rahane no run pushes it quick jus...,sports shreyas gopal drifter cricket in india ...
3,3,shreyas gopal to pietersen leg byes one run sp...,sports cut pietersen shot off leg foot cut sho...
4,4,shreyas gopal to rahane no run on the stumps n...,sports foot cut ravi shaz kp pitch over off sh...


In [17]:
sentences = Dataframe.iloc[:, 1].values
sentences = [str(i) for i in sentences]
sentences = [i.replace(',', ' ') for i in sentences]

tokens = tokenize_Words(sentences)

## TF-IDF functions


In [18]:
def tf(t, d):
    return float(d.count(t)) / len(d)

def idf(t):
    c_doc = 0
    for d in D:
        if t in d:
            c_doc += 1			
    return 0 if c_doc == 0 else math.log(len(D)/float(c_doc))

def tf_idf(q, d, op):
    if op=='and':	
        if len(set(q).intersection(d))==len(set(q)):
            ans = 0
            for t in q:
                ans += (tf(t, d) * idf(t))			
            return  ans
        else:
            return 0
    else:
        ans = 0
        for t in q:
            ans += (tf(t, d) * idf(t))
        return  ans

##Driver Code

In [ ]:
import math

while True:
    query = input('\nenter query string or \'q\' to quit:\n')

    if query=='q':
        break

    if query.startswith('"'):
        op='and'	
    else:
        op='or'	

    query=re.sub(r'[#+?!"/@$,]',' ',query)
    query = tokenize_Words([query])

    scores = dict()
    i = 0

    for d in tokens:
        sc=tf_idf(query[0],d,op)	
        if sc!=0:
            scores[i] = sc
        i += 1

    if len(scores)==0:
        print ('Matches not Found')
    else:
        top_docs=sorted(scores, key=scores.get, reverse=True)[:10]
        rank = 1
        for doc in top_docs:
            print (f'\nrank: {rank} score: {scores[doc]}')
            print( dataset_lines[doc].strip())
            rank+=1



enter query string or 'q' to quit:
pandya to rahane no run picks a short of length delivery from outside off and twirls his wrists but a good stop from rayudu at midwicket stops the single hardik pandya 
